# 練習環境  
- 限定されたサンプルを扱うなので、全サンプルから使えるサンプルだけのdatasetを作成する
- TrainのDatasetの各ＲＧＢチャンネルの平均と分散をとっておく

In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Meiryo'

### 画素の分布を確認

In [4]:
train_fn = r'dataset_cur_train.csv'
valid_fn = r'dataset_cur_test.csv'

In [5]:
df = pd.read_csv(train_fn,index_col=0)
display(df.shape , df.head(3))

(14949, 5)

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
8,1.0,0,True,3.0,./dataset_o\ECC2023_ResultsBook_Women_A-Divisi...
0,2.0,8,True,0.0,./dataset_o\ECC2021_ResultsBook_Men_A-Division...
8,-2.0,0,False,3.0,./dataset_o\ECC2019_ResultsBook_Women_A-Divisi...


In [10]:
df.sample(frac=0.03, random_state=2)['filepath'].values

array(['./dataset_o\\WMCC2023_ResultsBook\\geme503end4.png',
       './dataset_o\\ECC2019_ResultsBook_Men_A-Division\\geme313end8.png',
       './dataset_o\\CWC2018-19_Leg2_ResultsBook\\geme206end6.png',
       './dataset_o\\WMCC2023_ResultsBook\\geme452end7.png',
       './dataset_o\\WWCC2022_ResultsBook\\geme201end8.png',
       './dataset_o\\WWCC2022_ResultsBook\\geme418end1.png',
       './dataset_o\\WMCC2023_ResultsBook\\geme566end5.png',
       './dataset_o\\WWCC2022_ResultsBook\\geme330end9.png',
       './dataset_o\\OWG2018_ResultsBook\\geme780end5.png',
       './dataset_o\\WWCC2023_ResultsBook\\geme535end6.png',
       './dataset_o\\WWCC2018_ResultsBook\\geme842end6.png',
       './dataset_o\\OWG2022_ResultsBook\\geme983end3.png',
       './dataset_o\\WWCC2023_ResultsBook\\geme787end3.png',
       './dataset_o\\CWC2018-19_Leg3_ResultsBook\\geme343end2.png',
       './dataset_o\\ECC2019_ResultsBook_Men_A-Division\\geme365end8.png',
       './dataset_o\\WWCC2018_ResultsBook\\ge

In [12]:
srcimages = []
for path in df.sample(frac=0.01, random_state=2)['filepath'].values:
    image = Image.open(path)
    image_data = np.array(image)
    normalized_image = image_data.astype(np.float32)
    srcimages.append(normalized_image)
mean = np.mean(srcimages, axis=(0, 1, 2))
std = np.std(srcimages, axis=(0, 1, 2))
display('mean',mean.astype(float) , 'std',std )
print("ほどんど白なのでＲＧＢ全部1に近いはずなのに...この計算は没。")

'mean'

array([177.93385315, 177.93385315, 177.93385315])

'std'

array([75.60956, 75.58063, 76.43926], dtype=float32)

ほどんど白なのでＲＧＢ全部1に近いはずなのに...この計算は没。


In [13]:
mean1 = np.mean(srcimages[0:100], axis=(0, 1, 2))
mean2 = np.mean(srcimages[100:200], axis=(0, 1, 2))
mean3 = np.mean(srcimages[0:200], axis=(0, 1, 2))
display(mean1 , mean2 , mean3)

array([247.75   , 246.44637, 248.53317], dtype=float32)

array([245.11403, 243.26357, 247.99861], dtype=float32)

array([177.93385, 177.93385, 177.93385], dtype=float32)

#　計算を改良その１

In [14]:
srcimages = []
for path in df.sample(frac=0.01, random_state=2)['filepath'].values:
    image = Image.open(path)
    image_data = np.array(image)
    normalized_image = image_data.astype(np.float32)
    srcimages.append(normalized_image)
pictar = np.array( srcimages ) 

In [15]:
display(pictar.shape,len(pictar[:,:,:,0]))

display(['R:G:B',pictar[:,:,:,0].std()] )
display([ pictar[:,:,:,0].mean() , pictar[:,:,:,1].mean() , pictar[:,:,:,2].mean() ])
display([ pictar[:,:,:,0].std() , pictar[:,:,:,1].std() ,pictar[:,:,:,2].std() ])

(149, 540, 300, 3)

149

['R:G:B', 41.296883]

[241.10713, 238.94666, 246.62808]

[41.296883, 43.975304, 42.506752]

#　計算を改良その2

In [16]:
srcimages = []
for path in df.sample(frac=0.01, random_state=2)['filepath'].values:
    image = Image.open(path)
    image_data = np.array(image)
    normalized_image = image_data.astype(np.float32) / 255
    srcimages.append(normalized_image)
pictar = np.array( srcimages ) 
display(pictar.shape,len(pictar[:,:,:,0]))

display(['R:G:B',pictar[:,:,:,0].std()] )
display([ pictar[:,:,:,0].mean() , pictar[:,:,:,1].mean() , pictar[:,:,:,2].mean() ])
display([ pictar[:,:,:,0].std() , pictar[:,:,:,1].std() ,pictar[:,:,:,2].std() ])

(149, 540, 300, 3)

149

['R:G:B', 0.16194811]

[0.9455182, 0.9370455, 0.967168]

[0.16194811, 0.17245254, 0.16669254]

その２の値は妥当っぽい。ほぼ白だしなので１に近く、分散も低い  
しかし本番では、正方形にするため、余白を作ってさらにセンタークリップを行うので条件は変わる。
本番の変換を模索してみる

In [17]:
from torchvision import transforms
import torchvision
import torch

In [18]:
## 余白なしで計算をしてみる
target_size = 224
transform = transforms.Compose([
    #transforms.Pad(( 240 // 2, 0), fill=255, padding_mode='constant'),  # 左右に余白を追加
    #transforms.Resize(target_size),
    #transforms.CenterCrop(target_size),
    transforms.ToTensor()
])

In [19]:
srcimages = []
for path in df.sample(frac=0.01, random_state=2)['filepath'].values:
    image = Image.open(path)
    q = transform(image)
    q = q.unsqueeze(dim=0)
    srcimages.append(q)
tensor1 = torch.cat(srcimages)
display(tensor1.shape)
display(
    [tensor1[:,0,:,:].mean().item() ,tensor1[:,1,:,:].mean().item(),tensor1[:,2,:,:].mean().item()],
    [tensor1[:,0,:,:].std().item() ,tensor1[:,1,:,:].std().item(),tensor1[:,2,:,:].std().item()] )

torch.Size([149, 3, 540, 300])

[0.9455181956291199, 0.9370459318161011, 0.967168390750885]

[0.16194817423820496, 0.17245274782180786, 0.16669318079948425]

numpyでの計算に一致したので、本番の変換をかけて算出する

In [20]:
## 余白あり
target_size = 224
transform = transforms.Compose([
    transforms.Pad(( 240 // 2, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.ToTensor()
])
srcimages = []
for path in df.sample(frac=0.05, random_state=2)['filepath'].values:
    image = Image.open(path)
    q = transform(image)
    q = q.unsqueeze(dim=0)
    srcimages.append(q)
tensor1 = torch.cat(srcimages)
display(tensor1.shape)
display(
    [tensor1[:,0,:,:].mean().item() ,tensor1[:,1,:,:].mean().item(),tensor1[:,2,:,:].mean().item()],
    [tensor1[:,0,:,:].std().item() ,tensor1[:,1,:,:].std().item(),tensor1[:,2,:,:].std().item()] )

torch.Size([747, 3, 224, 224])

[0.5254706740379333, 0.5207158327102661, 0.5373605489730835]

[0.47727182507514954, 0.4752059578895569, 0.4878864884376526]

適当そうなので、自前の学習の時はこれを使う